# Using mit-bih-noise-stress-test-database, Test our models and index performances

## Import

In [ ]:
from petastorm import make_reader
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import os
import sys
import xarray as xr
sys.path.append(os.path.join(os.getcwd(), ".."))
from operations.dataset_manager import get_path_petastorm_format

## Set matplotlib window size and parameter

In [ ]:
plt.rcParams.update({"font.size": 32})
plt.rcParams["legend.fontsize"] = 32

## Paths

In [ ]:
models_save_path = "/workspaces/ecg_evaluation/results"
data_save_path = get_path_petastorm_format("mit-bih-noise-stress-test-database-1.0.0","ParquetFile")
save_path  = "/workspaces/ecg_evaluation/results"

# Get the dataset

In [ ]:
data = []
with make_reader(data_save_path) as reader:
    for row in reader:
        data.append(row)

In [ ]:
## get save files (where model is saved)

model_name = [
                f.split(".")[0]
                for f in os.listdir(models_save_path)
                if os.path.isfile(os.path.join(models_save_path,f )) and (f.endswith(".sav"))
            ]

In [ ]:
name_index = [
        "Corr_interlead",
        "Corr_intralead",
        "wPMF",
        "SNRECG",
        "HR",
        "Flatline",
        "TSD",
    ]

# Test the model on new dataset

## Divide into the 3 noises present in the dataset (em,ma,bw)

In [ ]:
if (save_path is not None) and (not os.path.exists(save_path)):
        os.makedirs(save_path)

if "file://" not in data_save_path:
   path_petastorm = f"file:///{data_save_path}"
else:
   path_petastorm = data_save_path


array_signal = []
array_name = []
array_fs = []
array_signal_name = []
array_units = []
array_nb_sig = []
array_nb_time_window = []
with make_reader(data_save_path) as reader:
    for sample in reader:
        if len(sample.signal[0,0,:]) != 5000:
            continue

        array_signal.append(sample.signal.reshape((sample.nb_time_window*sample.n_sig,sample.sig_len)))
        array_name.append(sample.noun_id.decode("utf-8"))
        array_fs.append(sample.fs)
        array_signal_name.append([sample.sig_name[j].decode("utf-8") for j in range(len(sample.sig_name))])
        array_units.append([sample.units[j].decode("utf-8") for j in range(len(sample.units))])
        array_nb_sig.append(sample.n_sig)
        array_nb_time_window.append(sample.nb_time_window)

ds_ecg = xr.Dataset(
        data_vars=dict(
            signal=(["id", "n_sig*nb_time_window", "time"], np.array(array_signal)),
            fs=(["id"], np.array(array_fs)),
            signal_names=(["id","signal_names"], np.array(array_signal_name)),
            units = (["id","units"],np.array(array_units)),
            n_time_window = (["id"],np.array(array_nb_time_window)),
            n_sig=(["id"], np.array(array_nb_sig))
        ),
        coords=dict(
            id=(["id"], np.array(array_name)),
            time=(["time"], np.arange(0, 5000)),

        ),
        attrs=dict(description="ecg with real noise data")
    )

if save_path is not None:
    ds_ecg.to_netcdf(os.path.join(save_path, "ecg_data_mit_bih_noise_test.nc"))


In [ ]:
print(ds_ecg.signal.values.shape)

## Test each model performance for each noise